# 核心模块组成
## 视觉编码器（Vision Encoder）
采用模型：ViT（Vision Transformer，如 ViT-G/14）。
作用：从原始图像中提取丰富的视觉语义特征。
具体过程：使用 Conv2d 将图像划分为 patch，经 39 层 Transformer Encoder 处理，输出尺寸为[batch_size, num_patches + 1, 1408]。
状态：完全冻结，不参与训练。
## Q-Former（Querying Transformer）
定位：连接视觉特征与语言模型的桥接模块，是可训练的轻量级 Transformer（参数量约 110M-188M）。
输入：32 或 64 个可学习的 Query Tokens（初始化为可训练参数）。
结构：每层包含自注意力（Self-Attention）与交叉注意力（Cross-Attention）。
自注意力：作用于[Q; T]（Q 为 Query Tokens，T 为文本 Tokens），受 attention mask 控制，用于独立编码图像和文本信息。
交叉注意力：仅作用在 Q 上，Query 对 ViT 输出的 visual token 做交叉注意力，将视觉特征从 1408 维投影为 768 维。
输出：尺寸为[batch, num_query_tokens, 768]的特征，包含与文本相关的关键视觉信息。
## 语言投影层（Language Projection）
作用：将 Q-Former 输出的 768 维特征映射为语言模型所需的维度（如 2560 维），实现维度适配。
状态：可训练。
## 语言模型（Language Model, LLM）
采用模型：如 OPT、Flan-T5 等（如 OPT 为 32 层 Decoder 结构）。
作用：进行最终的文本生成（如描述、问答、对话等），支持 causal masking。
输入：Q-Former 输出经投影后的特征和文本 prompt。
输出：预测词的概率分布。
状态：冻结或部分微调。

# 整体流程
图像经 ViT 提取视觉 token（1408 维）。
可学习 Query token 输入 Q-Former，与视觉 token 融合，输出 768 维特征。
768 维特征经 Linear 投影为语言模型所需维度（如 2560 维），作为输入前缀送入 LLM。
OPT 等 LLM 的 Decoder 进行文本生成（如 Caption、VQA、对话）。